In [1]:
"""
Changes made from 1st version: 
1) Changed the starting stitch distance to 1800 to account for the larger faces needing stitching Done
2) Fixed the stitch function error that was using an index number instead of vertex number Done
3) Fixed stitching function so when reverses order still starts on the original vertex Done
4) Made it so that function has "load_meshes_flag=True" and "save_meshes_flag=False" Done
--> for time saver



"""

'\nChanges made from 1st version: \n1) Changed the starting stitch distance to 1800 to account for the larger faces needing stitching Done\n2) Fixed the stitch function error that was using an index number instead of vertex number Done\n3) Fixed stitching function so when reverses order still starts on the original vertex Done\n4) Made it so that function has "load_meshes_flag=True" and "save_meshes_flag=False" Done\n--> for time saver\n\n\n\n'

In [2]:
from Stitcher_vp2 import stitch_neuron
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time
import os

In [3]:
# testing which table to pull from
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")

Connecting celiib@10.28.0.34:3306


# Check the datajoint parent will be correct

In [4]:
#gets the possible segment ids
pinky.Mesh() & (pinky.AllenSoma() & "cell_class='excitatory'")

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices)
3,648518346341371119,669574,1316372,=BLOB=,=BLOB=
3,648518346349386137,85806,166966,=BLOB=,=BLOB=
3,648518346349470171,1254494,2496165,=BLOB=,=BLOB=
3,648518346349471156,865120,1721566,=BLOB=,=BLOB=
3,648518346349471500,803277,1599182,=BLOB=,=BLOB=
3,648518346349471562,1343736,2654580,=BLOB=,=BLOB=
3,648518346349471565,2232110,4423873,=BLOB=,=BLOB=


In [5]:
@schema
class ExcitatoryStitchedMeshVp2(dj.Computed):
    definition="""
    -> pinky.Mesh
    ---
    n_vertices           : bigint           # number of vertices in this mesh
    n_triangles          : bigint           # number of triangles in this mesh
    vertices             : longblob         # x,y,z coordinates of vertices
    triangles            : longblob         # triangles (triplets of vertices)
    n_pieces             : int              # number of unconnected mesh pieces outside the largest mesh piece
    largest_piece_perc   : decimal(6,5)     # number of faces percentage of largest mesh piece in respect to total mesh
    outside_perc         : decimal(6,5)     # number of faces percentage of mesh outside the biggest mesh piece
    n_stitched           : int              # number of mesh pieces stitched back to main mesh
    stitched_addon_perc  : decimal(6,5)     # number of faces percentage of pieces that were stitched back in respect to largest mesh piece
    n_unstitched         : int              # number of mesh pieces remaining unstitched back to main mesh        
    unstitched_perc     : decimal(6,5)     # number of faces percentage of pieces that were not in respect to largest mesh piece
    
    
    """
    
    key_source = pinky.Mesh() & ((pinky.AllenSoma() & "cell_class='excitatory'").proj())
    
    
    def make(self, key):
        
        segment_id = key["segment_id"]
        key = dict(segment_id=segment_id,segmentation=3)
        vertices,triangles = (pinky.Mesh & key).fetch("vertices","triangles")

        verts = vertices[0]
        faces = triangles[0]

        print(f"\n\n\n*****Running Neuron Stitcher for {segment_id}**********")
        
        load_file_check = str(segment_id) + "_30_main_and_child_meshes_array.npz"
        files_to_list = os.listdir("./stitch_mesh_saved/")
        
        #check to see if file exists:
        if load_file_check in files_to_list:
            print("Using a load file for meshes")
            [n_vertices,
             n_triangles,
             vertices,
             triangles,
             n_pieces,
             largest_piece_perc,
             outside_perc,
             n_stitched,
             stitched_addon_perc,
             n_unstitched,
             unstitched_perc] = stitch_neuron(segment_id=segment_id,
                                                          vertices=verts,
                                                          faces=faces,
                                                        import_from_off=False,
                                                         load_meshes_flag = True,
                                                     save_meshes_flag = False,
                                                        pymeshfix_flag = False
                                                       )
        else:
        
            print("NOT Using a load file for meshes")

            [n_vertices,
             n_triangles,
             vertices,
             triangles,
             n_pieces,
             largest_piece_perc,
             outside_perc,
             n_stitched,
             stitched_addon_perc,
             n_unstitched,
             unstitched_perc] = stitch_neuron(segment_id=segment_id,
                                                          vertices=verts,
                                                          faces=faces,
                                                        import_from_off=False,
                                                         load_meshes_flag = False,
                                                     save_meshes_flag = True,
                                                        pymeshfix_flag = False
                                                       )

        #insert dummy dictionary into correspondence table
        insert_key = dict(key,
                          n_vertices=n_vertices,
                         n_triangles=n_triangles,
                         vertices=vertices,
                         triangles=triangles,
                         n_pieces=n_pieces,
                         largest_piece_perc=largest_piece_perc,
                         outside_perc=outside_perc,
                         n_stitched=n_stitched,
                         stitched_addon_perc=stitched_addon_perc,
                         n_unstitched=n_unstitched,
                         unstitched_perc=unstitched_perc)
                        
        
        self.insert1(insert_key,skip_duplicates=True)
        
        
                         

In [6]:
start = time.time()
ExcitatoryStitchedMeshVp2.populate(reserve_jobs=True)
print(time.time() - start)




*****Running Neuron Stitcher for 648518346349473835**********
Using a load file for meshes
import child and main meshes fromstitch_mesh_saved/648518346349473835_30_main_and_child_meshes_array.npz
Total time for Mesh Cleansing: 1.5400969982147217
Number of outside child meshes =  31
Main mesh and outside child meshes make up 0.89686596648891% of original mesh
 face length 905518 using optimized facets with 3 neighbors
Filtering edges by size finished: 27.843798875808716, facet # = 48734
Filtering by convexity and getting centers took: 37.09702682495117, facet # = 8637
Finished 8637 facets for main mesh: 66.44859218597412
Filtering edges by size finished: 0.001980304718017578, facet # = 2
Had to restructure the array because was 2D array
Filtering by convexity and getting centers took: 0.0019392967224121094, facet # = 2
Filtering edges by size finished: 0.001552581787109375, facet # = 2
Filtering by convexity and getting centers took: 0.0013740062713623047, facet # = 1
Filtering edges

In [7]:
#(schema.jobs & "table_name='__excitatory_stitched_mesh_vp2'").delete()
#schema.jobs

In [8]:
"""
Things didn't think about: 
1) if there are no child meshes to be stitched 
2) If there is only one mesh


"""

"\nThings didn't think about: \n1) if there are no child meshes to be stitched \n2) If there is only one mesh\n\n\n"

In [9]:
# import os
# files_to_list = os.listdir("./stitch_mesh_saved/")

# str(segment_id) + "_30_main_and_child_meshes_array.npz"
# "648518346349470171_30_main_and_child_meshes_array.npz" in files_to_list